<a href="https://colab.research.google.com/github/claudiosegala/Monografia/blob/master/code/tcc_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import google as g # To connect with google drive
g.colab.drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install numpy
!pip install pandas

In [0]:
import pandas as pd # data manipulation library
import numpy as np # math library
import datetime as dt # to discover week day

In [0]:
TCC_FOLDER = '/content/drive/My Drive/TCC/'

In [0]:
DATASET_PATH = f"{TCC_FOLDER}dataset/all_data_sorted.csv"

In [0]:
col_names = [
  'Sensor',
  'Date',
  'Time',
  'Lane',
  'Speed',
  'Max Speed',
  'Size'
]

data = pd.read_csv(DATASET_PATH, ';', header=None, names=col_names)

In [8]:
data.head()

,Sensor,Date,Time,Lane,Speed,Max Speed,Size
0,RSI128,2016/05/01,00:00:09,1,26.0,60.0,0.0
1,RSI131,2016/05/01,00:00:09,2,20.0,60.0,1.1
2,RSI132,2016/05/01,00:00:09,1,45.0,60.0,0.0
3,RSI131,2016/05/01,00:00:10,1,40.0,60.0,0.5
4,RSI129,2016/05/01,00:00:12,1,35.0,60.0,0.0


In [9]:
data.describe()

,Lane,Speed,Max Speed,Size
count,1.080178e+07,1.080178e+07,10801781.0,1.080178e+07
mean,1.656316e+00,3.654532e+01,60.0,1.049269e-01
std,7.029681e-01,1.133506e+01,0.0,8.707685e-01
min,1.000000e+00,1.000000e+00,60.0,0.000000e+00
25%,1.000000e+00,2.800000e+01,60.0,0.000000e+00
50%,2.000000e+00,3.700000e+01,60.0,0.000000e+00
75%,2.000000e+00,4.600000e+01,60.0,0.000000e+00
max,3.000000e+00,1.120000e+02,60.0,5.000000e+01


In [10]:
print(f"It contains {len(data['Sensor'])} entries\n\n")

It contains 10801781 entries




In [11]:
print(f"This dataset contains {len(set(data['Sensor']))} sensors.")

for val in set(data['Sensor']):
  print(f"Sensor {val} has {len(data[data['Sensor'] == val])}")

This dataset contains 8 sensors.
Sensor RSI128 has 536879
Sensor RSI018 has 2029559
Sensor RSI032 has 2117820
Sensor RSI033 has 2382754
Sensor RSI017 has 1686900
Sensor RSI132 has 652998
Sensor RSI131 has 816219
Sensor RSI129 has 578652


In [0]:
# Limit sensor usage
data = data[data['Sensor'] == 'RSI128']

In [0]:
# Remove unnecessary columns
data = data.drop(columns=['Sensor','Lane','Max Speed','Size'])

In [0]:
# Get datetime
data['Date'] = pd.to_datetime(data['Date'], format='%Y/%m/%d')

In [0]:
# Adjust type
f = lambda x : tm.strptime(x, '%H:%M:%S')
data['Time'] = data['Time'].apply(f)

g = lambda x : dt.timedelta(hours=x.tm_hour,minutes=x.tm_min,seconds=x.tm_sec).total_seconds()
data['Time'] = data['Time'].apply(g)

h = lambda x : int(x)
data['Time'] = data['Time'].apply(h)

In [0]:
# Create week day from date
j = lambda x : x.weekday()
data['WeekDay'] = data['Date'].apply(j)

In [17]:
data.head()

,Date,Time,Speed,WeekDay
0,2016-05-01,9,26.0,6
7,2016-05-01,18,32.0,6
39,2016-05-01,104,16.0,6
108,2016-05-01,206,22.0,6
160,2016-05-01,305,25.0,6


In [18]:
data.describe()

,Time,Speed,WeekDay
count,536879.000000,536879.000000,536879.000000
mean,51715.214387,31.078891,2.951632
std,17952.214406,7.168803,1.959146
min,3.000000,2.000000,0.000000
25%,38412.500000,26.000000,1.000000
50%,52364.000000,31.000000,3.000000
75%,65757.000000,35.000000,5.000000
max,86399.000000,67.000000,6.000000


In [19]:
for col, cont in data.iteritems():
  print(f"Column {col} has {cont.isnull().sum()} null elements")
  print(f"Column {col} has {cont.isna().sum()} nan elements")

Column Date has 0 null elements
Column Date has 0 nan elements
Column Time has 0 null elements
Column Time has 0 nan elements
Column Speed has 0 null elements
Column Speed has 0 nan elements
Column WeekDay has 0 null elements
Column WeekDay has 0 nan elements


In [20]:
start = data['Date'].min()
end = data['Date'].max()

print(f"\nThis data is from <{start}> to <{end}>. {(end - start).days + 1} days.\n")
print(f"It contains {len(data['Date'])} entries\n\n")


This data is from <2016-05-01 00:00:00> to <2016-07-31 00:00:00>. 92 days.

It contains 536879 entries




In [0]:
data.to_csv(f"{TCC_FOLDER}dataset/dataset.csv")